In [1]:
import math

from portfolio import Portfolio
from backtest import Backtest
from strategies import *
from config import *
from utils import plot_distribution
from data_processing import *

In [2]:
# get data
spy_df = get_spy_data()
options_df = generate_option_chain(spy_df, time_to_expiration, generate_at='open')

Time used for calculating option prices 4.2859 seconds.


In [3]:
# instantiation
my_portfolio = Portfolio(portfolio_initial_cash, portolio_weights_config)
my_strategy = ZeroCostCollar0DTE(my_portfolio, 'SPY', spy_df, options_df)
env = Backtest(my_portfolio, spy_df, options_df)

# data cleaning and calculation
env.main_df = my_strategy.select_and_update_options(env.main_df, strike_selection_config, options_df)
env.update_option_price_at_close()

In [4]:
# Buy and hold SPY at market open on Day 0
buy_date = env.main_df['Date'].values[0]
buy_price = env.main_df['Open'].values[0]

target_spy_exposure = my_portfolio.cash * my_portfolio.target_portfolio_weights['equity']
# n_spy_to_buy: buy SPY in multiples of option_contract_multiplier (typically 100) for easier hedging
n_spy_to_buy = math.floor(target_spy_exposure / buy_price / option_contract_multiplier) * option_contract_multiplier
my_strategy.execute_buy_and_hold_underlying('equity', buy_date, buy_price, n_spy_to_buy)

print('Current position:', my_portfolio.positions)
print('Current cash:', round(my_portfolio.cash, 2))
print('Transaction history')
my_portfolio.print_transaction_history()


Current position: {'equity': {'SPY': 2200}}
Current cash: 410752.01
Transaction history
1/2/18: ['bought 2200 SPY at 267.839996 on 1/2/18.']


In [5]:
# Run the zero-cost collar strategy every day in spy_df
env.run(my_strategy)

Exception: Not enough shorting power to short 2200 call K=304.0 at 1.0216102079118738 on 10/30/19. Available cash: 1591.9171610342762, Required magin: 2247.5424574061226, Leverage: 1

In [6]:
my_portfolio.cash


3722.795074403707

In [7]:
my_portfolio.print_transaction_history()

1/2/18: ['bought 2400 SPY at 267.839996 on 1/2/18.', 'shorted 2400 call K=269.0 at 0.6477275192673396 on 1/2/18.', 'bought 2400 put K=263.0 at 0.04437495258377984 on 1/2/18.', 'sold 2400 put K=263.0 at 0.0 on 1/2/18, remaining quantity: 0', 'bought 2400 call K=269.0 at 0.0 on 1/2/18.']
1/3/18: ['shorted 2400 call K=270.0 at 0.6942398385796474 on 1/3/18.', 'bought 2400 put K=264.0 at 0.04071542905233115 on 1/3/18.', 'sold 2400 put K=264.0 at 0.0 on 1/3/18, remaining quantity: 0', 'bought 2400 call K=270.0 at 0.47000100000002476 on 1/3/18.']
1/4/18: ['shorted 2400 call K=272.0 at 0.7934132164615768 on 1/4/18.', 'bought 2400 put K=267.0 at 0.08179442178944735 on 1/4/18.', 'sold 2400 put K=267.0 at 0.0 on 1/4/18, remaining quantity: 0', 'bought 2400 call K=272.0 at 0.0 on 1/4/18.']
1/5/18: ['shorted 2400 call K=273.0 at 0.9270119150339404 on 1/5/18.', 'bought 2400 put K=268.0 at 0.06463367804941827 on 1/5/18.', 'sold 2400 put K=268.0 at 0.0 on 1/5/18, remaining quantity: 0', 'bought 2400 c

In [ ]:
# Plot the distribution of collar strategy cost. Negative cost means the call premium is larger than put premium (i.e. receiving a credit to open)
plot_distribution(env.main_df['collar_cost'], x_label='Collar Cost Distribution', title='Cost of Collar')

In [ ]:
# Plot the distribution collar strategy PnL
plot_distribution(env.main_df['collar_pnl'], x_label='Daily Collar Profit', title='Collar Profit Distribution')

In [ ]:
plot_distribution(env.main_df['intraday_return'], x_label='Intraday Return during Trading Hours', title='Intraday Percentage Return Distribution')

In [ ]:
# save the main df in the backtest process to the data folder
env.save_main_df_to_csv(filename='backtest_main_df.csv')